**Hey there!** 

Thanks for stopping by and taking a look at this notebook. I would be really grateful if you could spend some time on the notebook and give me some areas of improvement. Looking forward to learning from you. 

**Cheers,
Komal**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from palettable.colorbrewer.qualitative import Pastel1_7
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from sklearn.metrics import f1_score

In [ ]:
df=pd.read_csv('/kaggle/input/travel-insurance/travel insurance.csv')
df.head()

In [ ]:
df.shape

*Dataset has 11 columns and 63326 rows. Out of the 11 columns, Claim is a target variable.*

In [ ]:
df.columns

All the column names are listed above. Since there is whitespace between the 2 word column names, let's rename the column names to avoid any naming issues.

In [ ]:
df.rename(columns={ 'Agency Type':'Agency_Type', 'Distribution Channel':'Distribution_Channel', 'Product Name':'Product_Name','Net Sales':'Net_Sales', 'Commision (in value)':'Commision'},inplace=True)

The necessary columns have been successfully renamed.

In [ ]:
df.info()

From the info, we know that there are 4 numerical columns while 7 are categorical.

In [ ]:
df.isnull().sum().any


In [ ]:
df=df.drop(['Gender'], axis=1)
df.head()

The Gender column has been dropped since majority of its values are null.

In [ ]:
column_keys=df.select_dtypes(include=['object']).columns.tolist()
for key in column_keys:
    print('Unique elements of',key,'are: ')
    print(df[key].unique(),end='\n')
    print(end='\n')

In [ ]:
df1 = df.groupby(by=["Destination"]).size().reset_index(name="counts")
df1.nlargest(15,['counts'])
df1['DestinationNew'] = np.where(df1['counts']>1090, df1['Destination'], 'Others')
fig = px.pie(df1, values='counts', names='DestinationNew', title='Popular Destinations among insured')
fig.show()

A pie chart depicting 15 most popular destinations among travel insured.Here, I have grouped countries which have less than 1090 insurance policies together into others.

In [ ]:
fig = plt.figure(figsize = (10, 5))
plt.hist(df['Age'])
plt.xlabel("Age")
plt.ylabel("Number of people")
plt.title("Distribution of Age")
plt.show()

The above figure shows a histogram depicting the distribution of age. From the graph, the majory of people opting for travel insurance are of the age 35 to 50.

In [ ]:
df2 = df.groupby(by=["Agency"]).size().reset_index(name="counts")
fig = plt.figure(figsize = (10, 5))
plt.bar(df2['Agency'], df2['counts'], color ='maroon', width = 0.4)
plt.xlabel("Agency")
plt.ylabel("Number of people insured")
plt.title("People insured in different Insurance Agencies")
plt.show()

The majority of people are insured under EPX agency.

In [ ]:
df3 = df.groupby(by=["Product_Name"]).size().reset_index(name="counts")
fig = px.pie(df3, values='counts', names='Product_Name', title='People insured under different plans')
fig.show()

The pie chart depicts that majority of travel insured people have opted for Cancellation Plan.

In [ ]:
df4=df.groupby(by=['Agency']).mean().reset_index()
df4.head()
fig = plt.figure(figsize = (10, 5))
plt.bar(df4['Agency'], df4['Commision'], color ='green', width = 0.4)
plt.xlabel("Agency")
plt.ylabel("Avg. Commision")
plt.title("Average commision of agents in various agency")
plt.show()

LWC is the agency that gives maximum commision to its agents while EPX gives NIL.

In [ ]:
fig = plt.figure(figsize = (10, 5))
plt.bar(df4['Agency'], df4['Net_Sales'], color ='pink', width = 0.4)
plt.xlabel("Agency")
plt.ylabel("Avg. Net Sales")
plt.title("Average Net Sales per agency")
plt.show()

The net sales are highest for LWC while lowest for SSI.

In [ ]:
df5=df.groupby(by=['Distribution_Channel']).size().reset_index(name="counts")
plt.bar(df5['Distribution_Channel'], df5['counts'], color ='purple',width = 0.4)
plt.xlabel("Distribution Channel")
plt.ylabel("Number of people insured")
plt.title("People insured under different Distribution Channel")
plt.show()

Majority of people have opted for online mode of insurance distribution channel.

In [ ]:
df6=df.loc[df['Claim']=='Yes']
df7= df6.groupby(by=["Destination"]).size().reset_index(name="counts")
df8=df7.nlargest(5,['counts'])
my_circle = plt.Circle((0, 0), 0.6, color='white')
plt.pie(df8['counts'], labels=df8['Destination'], autopct='%1.1f%%', colors=Pastel1_7.hex_colors)
plt.title('Top 5 countries where insured are raising claim requests')
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

Singapore tops the list with maximum number of claim request filed. 

In [ ]:
df9= df6.groupby(by=["Product_Name"]).size().reset_index(name="counts")
df10=df9.nlargest(5,['counts'])
my_circle = plt.Circle((0, 0), 0.6, color='white')
plt.pie(df10['counts'], labels=df10['Product_Name'], autopct='%1.1f%%',colors=Pastel1_7.hex_colors)
plt.title('Top 5 plans where insured are raising claim requests')
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

The majority claim requests are coming from Bronze Plan and Annual Silver Plan.

In [ ]:
df11= df6.groupby(by=["Agency"]).size().reset_index(name="counts")
df12=df11.nlargest(5,['counts'])
my_circle = plt.Circle((0, 0), 0.6, color='white')
plt.pie(df12['counts'], labels=df12['Agency'], autopct='%1.1f%%', colors=Pastel1_7.hex_colors)
plt.title('Top 5 Agencies where insured are raising claim requests')
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

The maximum claim requests have come to the agency called C2B.

In [ ]:
df13=df.groupby(by=['Agency_Type']).size().reset_index(name="counts")
plt.bar(df13['Agency_Type'], df13['counts'], color ='red',width = 0.4)
plt.xlabel("Agency Type")
plt.ylabel("Number of people insured")
plt.title("People insured under different Agency Type")
plt.show()

Majority of people have got their insurance via Travel Agency.

In [ ]:
df14 = df.groupby(by=["Claim"]).size().reset_index(name="counts")
plt.bar(df14['Claim'], df14['counts'], color ='grey',width = 0.4)
plt.xlabel("Claim")
plt.ylabel("Number of people insured")
plt.title("People Insured who requested for claim vs who did not")
plt.show()

Around 900 people have requested for claim out of total policy holders which is around 1.4%.

In [ ]:
fig = plt.figure(figsize =(6,4 ))
plt.boxplot(df['Duration'])
plt.show()

The boxplot shows data distribution of the Duration feature. We can see that there are some outliers depicting negative values of duration i.e. Time which ideally is not possible.

In [ ]:
index_duration = df[df['Duration'] < 0 ].index
df.drop(index_duration, inplace = True)

Dropped off negative values of Duration column.

In [ ]:
from sklearn import preprocessing
label_encoder1 = preprocessing.LabelEncoder()
df['Agency']= label_encoder1.fit_transform(df['Agency'])
  
label_encoder2 = preprocessing.LabelEncoder()
df['Agency_Type']= label_encoder2.fit_transform(df['Agency_Type'])

label_encoder3 = preprocessing.LabelEncoder()
df['Distribution_Channel']= label_encoder3.fit_transform(df['Distribution_Channel'])

label_encoder4 = preprocessing.LabelEncoder()
df['Product_Name']= label_encoder4.fit_transform(df['Product_Name'])

label_encoder5 = preprocessing.LabelEncoder()
df['Claim']= label_encoder5.fit_transform(df['Claim'])


label_encoder6 = preprocessing.LabelEncoder()
df['Destination']= label_encoder6.fit_transform(df['Destination'])


Label encoding is used to transform categorical features.

In [ ]:
df

In [ ]:
column_names = ["Agency", "Agency_Type", "Distribution_Channel","Product_Name","Duration","Destination","Net_Sales","Commision","Age","Claim"]
df = df.reindex(columns=column_names)

Reordered the columns of the dataset to keep the target column i.e. Claim at the end.

In [ ]:
plt.figure(figsize = (20,8))
sns.heatmap(df.corr(), square=True,annot=True,cmap= 'coolwarm')

Plotted the corelation coefficient to check the degree of colinearity which each individidual feature have with the output label. Clearly there is no high colinearity among any of the features. 

In [ ]:
y=df.iloc[:,[-1]]
X=df.drop(y.columns,axis = 1)

Sepearting features and target variable into X and y respectively.

In [ ]:
X

In [ ]:
y

In [ ]:
sm = SMOTE(random_state=42)
X_sm, y_sm = sm.fit_resample(X, y)
print(f'''Shape of X before SMOTE: {X.shape}
Shape of X after SMOTE: {X_sm.shape}''')
print('\nBalance of positive and negative classes (%):')
y_sm.value_counts(normalize=True) * 100

SMOTE is performed to handle imbalance target class by oversampling.

In [ ]:
X_sm

In [ ]:
y_sm

In [ ]:
X_new=pd.DataFrame()
to_scale = X_sm.columns
mms = MinMaxScaler()
X_new[to_scale] = mms.fit_transform(X_sm[to_scale])
X_new.head()

Scaling is performed to keep the values in range of 0 and 1.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y_sm, test_size=0.25, random_state=42)

The entire dataset is split into Training and test dataset.

In [ ]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
preds = model.predict(X_test)


Looking at the dataset, its clear that the data is not linear. Hence, Random Forest Classifier should give some prominent results. 

In [ ]:
print(f'Accuracy = {accuracy_score(y_test, preds):.2f}\nRecall = {recall_score(y_test, preds):.2f}\n')
cm = confusion_matrix(y_test, preds)
plt.figure(figsize=(8, 6))
plt.title('Confusion Matrix', size=16)
sns.heatmap(cm, annot=True, cmap='Blues');

In [ ]:
f1_score(y_test, preds, average='macro')

## A good accuracy of 97% and an f1 score of 0.97 is achieved through the model.